In [5]:
import glob
from pathlib import Path  
import numpy as np
import pandas as pd
# import geopandas
from shapely.geometry import Point

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [35]:
berkeley_data = pd.read_csv('formatted_allstops_large(expanded) (1).csv')
census_data_multi = pd.read_csv('census_info_multilinear(with distance).csv')
# census_tracts = geopandas.read_file('Census Tract Polygons 2010 (With Income+Race+Numstops Data).geojson')

# berkeley_data = berkeley_data.rename(columns={'censustract': 'name10'})

C:\Users\kharr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (23,24) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [23]:
census_data_multi.rename(columns={'name10':'censustract'}, inplace=True)
census_data_multi = census_data_multi[['censustract', 'lat', 'lon', 'medianincome',
        'totalpop', 'whitepop', 'nonwhitepop', 'pocpop', 'nonwhitecomp', 'poccomp',
        'aapop', 'na_aipop', 'aisianpop', 'hawaiian',
        'mixed2', 'totalnumstops', 'annualstops',
        'distancefromcal']]
census_data_multi.head()

,censustract,lat,lon,medianincome,totalpop,whitepop,nonwhitepop,pocpop,nonwhitecomp,poccomp,aapop,na_aipop,aisianpop,hawaiian,mixed2,totalnumstops,annualstops,distancefromcal
0,4211.0,37.899434,-122.266193,206199.0,1992,1853,288.0,106.0,0.069779,0.053213,41.0,0.0,182.0,9.0,56.0,111,14,1.944982
1,4212.0,37.897548,-122.275227,198576.0,3563,2504,1142.0,441.0,0.297221,0.123772,34.0,4.0,701.0,0.0,403.0,202,25,1.991248
2,4213.0,37.891109,-122.281366,83214.0,3844,3204,793.0,405.0,0.166493,0.105359,58.0,6.0,388.0,4.0,337.0,280,35,1.821434
3,4214.0,37.888341,-122.270922,180625.0,1593,1325,303.0,179.0,0.168236,0.112367,28.0,7.0,124.0,4.0,140.0,160,20,1.321686
4,4215.0,37.892035,-122.259031,190484.0,3640,3102,608.0,213.0,0.147802,0.058516,123.0,0.0,395.0,11.0,79.0,112,14,1.388959


In [16]:
berkeley_data['Date of Stop'] = pd.to_datetime(berkeley_data['Date of Stop'])
berkeley_data['mm'] = berkeley_data['Date of Stop'].dt.month
berkeley_data['yyyy'] = berkeley_data['Date of Stop'].dt.year

berkeley_data = berkeley_data.iloc[:,1:]
berkeley_data.head()

,LEA Record ID,Incident Number,Date of Stop,Time of Stop,Duration of Stop,City,LAT,LONG,Race Perceived Prior To Stop,Perceived Race or Ethnicity,...,censustract,forceused,distancefromcal,mm,yyyy,area_totalpop,area_medianincome,area_totalstops,area_annualstops,nonwhitecomp
0,BPD000021033,NaN,2020-10-01,11:14,8.0,Berkeley,37.855313,-122.266571,True,Hispanic/Latino,...,4235.0,0,1.2,10,2020,3118.0,71352.0,3486,436.0,0.377806
1,BPD000021062,NaN,2020-10-02,15:21,7.0,Berkeley,37.854378,-122.271169,True,Black/African American,...,4235.0,1,1.4,10,2020,3118.0,71352.0,3486,436.0,0.377806
2,BPD000021048,NaN,2020-10-02,5:04,10.0,Berkeley,37.859469,-122.266941,False,White,...,4235.0,0,1.0,10,2020,3118.0,71352.0,3486,436.0,0.377806
3,BPD000021083,NaN,2020-10-03,18:57,23.0,Berkeley,37.855689,-122.269154,True,White,...,4235.0,0,1.3,10,2020,3118.0,71352.0,3486,436.0,0.377806
4,BPD000021126,NaN,2020-10-03,19:56,30.0,Berkeley,37.857611,-122.267249,True,Black/African American,...,4235.0,1,1.1,10,2020,3118.0,71352.0,3486,436.0,0.377806


In [17]:
berkeley_data['censustract'].unique().size

33

In [31]:
by_census = berkeley_data[['name10']].copy()
by_census.shape
by_census = by_census.groupby('name10').size().to_frame(name='All Time Stop Count')
by_census.shape
by_census['Annual Average Stop Count'] = round((by_census['All Time Stop Count']/8),0)

by_census.head(50)
# by_census = by_census.groupby('name10')
# by_census.shape
# .size().to_frame(name = 'Count').sort_values("Count", ascending = False).head(10).reset_index()
# by_census.rename(columns={'Count':'Total Crimes'})

(61702, 1)

(33, 1)

,All Time Stop Count,Annual Average Stop Count
name10,,
4211.00,111,14.0
4212.00,202,25.0
4213.00,280,35.0
4214.00,160,20.0
4215.00,112,14.0
4216.00,87,11.0
4217.00,677,85.0
4218.00,524,66.0
4219.00,1597,200.0


In [32]:
df = pd.merge(census_tracts, by_census, how='left', left_on='name10', right_on='name10')
df.columns = df.columns.str.replace(' ','_').str.lower()
df.columns = df.columns.str.replace('__','').str.lower()
df = df.drop(columns=['numstops'])
df = df.rename(columns={'all_time_stop_count':'totalnumstops','annual_average_stop_count':'annualstops'})
df['nonwhitecomp'] = 0

for i in range(df.index.size):
    if i == 30:
        continue
    totalpop = float(df.loc[i,'totalpop'].replace(',',''))
    whitepop = float(df.loc[i,'whitepop'].replace(',',''))
    df.loc[i,'nonwhitecomp'] = (totalpop-whitepop)/totalpop

In [41]:
df.head(2)
df.columns.unique

,stroke,stroke-width,stroke-opacity,fill,fill-opacity,mtfcc10,name10,geo,totalpop,intptlat10,...,whitepop,aapop,na_aipop,aisianpop,hawaiian,mixed2,geometry,totalnumstops,annualstops,nonwhitecomp
0,#555555,2,1,#555555,0.5,G5020,4238.0,"Census Tract 4238, Alameda County, California",2925.0,+37.8547587,...,"2,626.0",71.0,0.0,269.0,0.0,154.0,"POLYGON ((-122.24192 37.85202, -122.24180 37.8...",739,92.0,0.102222
1,#555555,2,1,#555555,0.5,G5020,4222.0,"Census Tract 4222, Alameda County, California",3144.0,+37.8728822,...,2125.0,151.0,29.0,405.0,4.0,159.0,"POLYGON ((-122.29365 37.87345, -122.29359 37.8...",3346,418.0,0.324109


<bound method Index.unique of Index(['stroke', 'stroke-width', 'stroke-opacity', 'fill', 'fill-opacity',
       'mtfcc10', 'name10', 'geo', 'totalpop', 'intptlat10', 'awater10',
       'namelsad10', 'id', 'aland10', 'geoid10', 'tractce10', 'intptlon10',
       'medianincome', 'whitepop', 'aapop', 'na_aipop', 'aisianpop',
       'hawaiian', 'mixed2', 'geometry', 'totalnumstops', 'annualstops',
       'nonwhitecomp'],
      dtype='object')>

In [50]:
berkeley_data_for_stata = berkeley_data.copy()

expanded_berkeley_data = pd.merge(berkeley_data_for_stata, df, how='left', left_on='name10', right_on='name10')
expanded_berkeley_data.head(5)
expanded_berkeley_data.shape
# expanded_berkeley_data['nonwhitecomp_y']

,LEA Record ID,Incident Number,Date of Stop,Time of Stop,Duration of Stop,City,LAT,LONG,Race Perceived Prior To Stop,Perceived Race or Ethnicity,...,whitepop,aapop,na_aipop,aisianpop,hawaiian,mixed2,geometry,totalnumstops,annualstops,nonwhitecomp
0,BPD000021033,NaN,2020-10-01,11:14,8.0,Berkeley,37.855313,-122.266571,True,Hispanic/Latino,...,1940.0,585.0,11.0,434.0,6.0,422.0,"POLYGON ((-122.26471 37.86157, -122.26474 37.8...",3486,436.0,0.377806
1,BPD000021062,NaN,2020-10-02,15:21,7.0,Berkeley,37.854378,-122.271169,True,Black/African American,...,1940.0,585.0,11.0,434.0,6.0,422.0,"POLYGON ((-122.26471 37.86157, -122.26474 37.8...",3486,436.0,0.377806
2,BPD000021048,NaN,2020-10-02,5:04,10.0,Berkeley,37.859469,-122.266941,False,White,...,1940.0,585.0,11.0,434.0,6.0,422.0,"POLYGON ((-122.26471 37.86157, -122.26474 37.8...",3486,436.0,0.377806
3,BPD000021083,NaN,2020-10-03,18:57,23.0,Berkeley,37.855689,-122.269154,True,White,...,1940.0,585.0,11.0,434.0,6.0,422.0,"POLYGON ((-122.26471 37.86157, -122.26474 37.8...",3486,436.0,0.377806
4,BPD000021126,NaN,2020-10-03,19:56,30.0,Berkeley,37.857611,-122.267249,True,Black/African American,...,1940.0,585.0,11.0,434.0,6.0,422.0,"POLYGON ((-122.26471 37.86157, -122.26474 37.8...",3486,436.0,0.377806


(61702, 82)

In [51]:
expanded_berkeley_data = expanded_berkeley_data.rename(columns={'totalnumstops':'area_totalstops',
                                                                'annualstops':'area_annualstops',
                                                                'totalpop':'area_totalpop',
                                                                'medianincome':'area_medianincome',
                                                                'name10':'censustract'})
expanded_berkeley_data = expanded_berkeley_data.drop(columns=['stroke', 'stroke-width', 'stroke-opacity', 'fill',
                                                              'fill-opacity', 'mtfcc10', 'geo','intptlat10', 'awater10',
                                                              'namelsad10', 'id', 'aland10', 'geoid10', 'tractce10', 'intptlon10','aapop', 'na_aipop', 'aisianpop',
                                                              'hawaiian', 'whitepop', 'mixed2', 'geometry'])
expanded_berkeley_data.columns.unique

<bound method Index.unique of Index(['LEA Record ID', 'Incident Number', 'Date of Stop', 'Time of Stop',
       'Duration of Stop', 'City', 'LAT', 'LONG',
       'Race Perceived Prior To Stop', 'Perceived Race or Ethnicity',
       'Perceived Gender', 'Perceived Age', 'Reason for Stop',
       'Reason for Stop Narrative', 'Result of Stop', 'Person Number',
       'Is Stop Made in Response to Call for Service',
       'Information Based Stop', 'Type Of Stop', 'Officer Type of Assignment',
       'Location', 'Is Location a K12 Public School',
       'If K12 School Is Stop of a Student', 'School Name',
       'Education Code Section', 'Education Code Subdivision',
       'Perceived Gender Nonconforming', 'Is LGBT',
       'Person had Limited or No English Fluency',
       'Perceived or Known Disability', 'City of Residence',
       'Traffic Violation Type', 'Traffic Violation Offense Codes',
       'Suspicion Offense Code', 'Suspicion SubType', 'Actions Taken',
       'Basis for Search', 

In [52]:
filepath = Path('formatted_allstops_large(expanded).csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
expanded_berkeley_data.to_csv(filepath, na_rep='')

In [ ]:
# df.to_file("Census Tract Polygons 2010 (With Income+Race+Numstops Data)(Expanded).geojson", driver='GeoJSON')


In [45]:
berkeley_data = berkeley_data[['LEA Record ID', 'Incident Number', 'Date of Stop', 'Time of Stop',
       'Duration of Stop', 'City', 'LAT', 'LONG',
       'Race Perceived Prior To Stop', 'Perceived Race or Ethnicity',
       'Perceived Gender', 'Perceived Age', 'Reason for Stop',
       'Reason for Stop Narrative', 'Result of Stop', 'Person Number',
       'Is Stop Made in Response to Call for Service',
       'Information Based Stop', 'Type Of Stop', 'Officer Type of Assignment',
       'Location', 'Is Location a K12 Public School',
       'If K12 School Is Stop of a Student', 'School Name',
       'Education Code Section', 'Education Code Subdivision',
       'Perceived Gender Nonconforming', 'Is LGBT',
       'Person had Limited or No English Fluency',
       'Perceived or Known Disability', 'City of Residence',
       'Traffic Violation Type', 'Traffic Violation Offense Codes',
       'Suspicion Offense Code', 'Suspicion SubType', 'Actions Taken',
       'Basis for Search', 'Basis for Search Narrative',
       'Basis for Property Seizure', 'Type of Property Seized',
       'Contraband or Evidence', 'Other Contraband Desc',
       'Warning Offense Codes', 'Citation Offense Codes',
       'In field cite and release Offense Codes',
       'Custodial arrest Offense Codes', 'IncidentNumber', 'Address',
       'CallType', 'Car Search', 'censustract']]
berkeley_data.head(2)
berkeley_data.shape

,LEA Record ID,Incident Number,Date of Stop,Time of Stop,Duration of Stop,City,LAT,LONG,Race Perceived Prior To Stop,Perceived Race or Ethnicity,...,Other Contraband Desc,Warning Offense Codes,Citation Offense Codes,In field cite and release Offense Codes,Custodial arrest Offense Codes,IncidentNumber,Address,CallType,Car Search,censustract
0,BPD000021033,NaN,2020-10-01,11:14,8.0,Berkeley,37.855313,-122.266571,True,Hispanic/Latino,...,NaN,NaN,VC 23123(A) USE CELLPH W/DRIV W/O HFD (I) | VC...,NaN,NaN,NaN,NaN,NaN,NaN,4235.0
1,BPD000021062,NaN,2020-10-02,15:21,7.0,Berkeley,37.854378,-122.271169,True,Black/African American,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4235.0


(61702, 51)

In [47]:
census_data_multi.rename(columns={ 'name10': 'censustract',
                                   'medianincome':'tract_medianincome', 
                                   'totalpop':'tract_totalpop', 'whitepop':'tract_whitepop',
                                   'nonwhitepop':'tract_nonwhitepop', 'pocpop':'tract_pocpop', 
                                   'nonwhitecomp':'tract_nonwhitecomp', 'poccomp':'tract_poccomp', 
                                   'aapop':'tract_aapop', 'na_aipop':'tract_na_aipop',
                                   'aisianpop':'tract_aisianpop', 'hawaiian':'tract_hawaiian', 
                                   'mixed2':'tract_mixed2', 'totalnumstops':'tract_totalnumstops', 
                                   'annualstops':'tract_annualstops',
                                   'distancefromcal':'tract_distancefromcal'}, inplace=True)
census_data_multi = census_data_multi[['censustract', 
                                       'tract_medianincome', 'tract_totalpop', 'tract_whitepop',
                                       'tract_nonwhitecomp', 'tract_poccomp', 'tract_nonwhitepop',
                                       'tract_pocpop', 'tract_aapop', 'tract_na_aipop', 'tract_aisianpop',
                                       'tract_hawaiian', 'tract_mixed2', 'tract_totalnumstops',
                                       'tract_annualstops', 'tract_distancefromcal']]
census_data_multi.head(2)
census_data_multi.shape

,censustract,tract_medianincome,tract_totalpop,tract_whitepop,tract_nonwhitecomp,tract_poccomp,tract_nonwhitepop,tract_pocpop,tract_aapop,tract_na_aipop,tract_aisianpop,tract_hawaiian,tract_mixed2,tract_totalnumstops,tract_annualstops,tract_distancefromcal
0,4211.0,206199.0,1992,1853,0.069779,0.053213,288.0,106.0,41.0,0.0,182.0,9.0,56.0,111,14,1.944982
1,4212.0,198576.0,3563,2504,0.297221,0.123772,1142.0,441.0,34.0,4.0,701.0,0.0,403.0,202,25,1.991248


(33, 16)

In [48]:
expanded = pd.merge(berkeley_data, census_data_multi, left_on='censustract',right_on='censustract')
expanded.head(2)
expanded.columns.unique

,LEA Record ID,Incident Number,Date of Stop,Time of Stop,Duration of Stop,City,LAT,LONG,Race Perceived Prior To Stop,Perceived Race or Ethnicity,...,tract_nonwhitepop,tract_pocpop,tract_aapop,tract_na_aipop,tract_aisianpop,tract_hawaiian,tract_mixed2,tract_totalnumstops,tract_annualstops,tract_distancefromcal
0,BPD000021033,NaN,2020-10-01,11:14,8.0,Berkeley,37.855313,-122.266571,True,Hispanic/Latino,...,1458.0,1024.0,585.0,11.0,434.0,6.0,422.0,3486,436,1.009877
1,BPD000021062,NaN,2020-10-02,15:21,7.0,Berkeley,37.854378,-122.271169,True,Black/African American,...,1458.0,1024.0,585.0,11.0,434.0,6.0,422.0,3486,436,1.009877


<bound method Index.unique of Index(['LEA Record ID', 'Incident Number', 'Date of Stop', 'Time of Stop',
       'Duration of Stop', 'City', 'LAT', 'LONG',
       'Race Perceived Prior To Stop', 'Perceived Race or Ethnicity',
       'Perceived Gender', 'Perceived Age', 'Reason for Stop',
       'Reason for Stop Narrative', 'Result of Stop', 'Person Number',
       'Is Stop Made in Response to Call for Service',
       'Information Based Stop', 'Type Of Stop', 'Officer Type of Assignment',
       'Location', 'Is Location a K12 Public School',
       'If K12 School Is Stop of a Student', 'School Name',
       'Education Code Section', 'Education Code Subdivision',
       'Perceived Gender Nonconforming', 'Is LGBT',
       'Person had Limited or No English Fluency',
       'Perceived or Known Disability', 'City of Residence',
       'Traffic Violation Type', 'Traffic Violation Offense Codes',
       'Suspicion Offense Code', 'Suspicion SubType', 'Actions Taken',
       'Basis for Search', 

In [55]:
expanded['Race Perceived Prior To Stop'] = expanded['Race Perceived Prior To Stop'].astype(str)
expanded.columns = expanded.columns.str.replace(' ','_').str.lower()
expanded.columns.unique

<bound method Index.unique of Index(['lea_record_id', 'incident_number', 'date_of_stop', 'time_of_stop',
       'duration_of_stop', 'city', 'lat', 'long',
       'race_perceived_prior_to_stop', 'perceived_race_or_ethnicity',
       'perceived_gender', 'perceived_age', 'reason_for_stop',
       'reason_for_stop_narrative', 'result_of_stop', 'person_number',
       'is_stop_made_in_response_to_call_for_service',
       'information_based_stop', 'type_of_stop', 'officer_type_of_assignment',
       'location', 'is_location_a_k12_public_school',
       'if_k12_school_is_stop_of_a_student', 'school_name',
       'education_code_section', 'education_code_subdivision',
       'perceived_gender_nonconforming', 'is_lgbt',
       'person_had_limited_or_no_english_fluency',
       'perceived_or_known_disability', 'city_of_residence',
       'traffic_violation_type', 'traffic_violation_offense_codes',
       'suspicion_offense_code', 'suspicion_subtype', 'actions_taken',
       'basis_for_search', 

In [64]:
expanded.to_csv('formatted_allstops_large(expanded) (2).csv')

In [78]:
test = df[df['censustract']!='']
test.shape

(8090, 47)

In [84]:
df = pd.read_stata('C:\\Users\\kharr\\Documents\\GitHub\\Berkeley-PD-ISF-110\\Data\\formatted_allstops.dta')  
print('df is',df.shape,'big.')
df = df[['dateofstop', 'mm', 'yyyy', 'timeofstop', 'durationofstop',
       'isstopmadeinresponsetocallforser', 'informationbasedstop',
       'typeofstop', 'officertypeofassignment', 'location', 'city',
       'islocationak12publicschool', 'ifk12schoolisstopofastudent',
       'schoolname', 'educationcodesection', 'educationcodesubdivision',
       'raceperceivedpriortostop', 'perceivedraceorethnicity', 'gender',
       'perceivedgendernonconforming', 'islgbt', 'perceivedage',
       'personhadlimitedornoenglishfluen', 'perceivedorknowndisability',
       'cityofresidence', 'reasonforstop', 'reasonforstopnarrative',
       'trafficviolationtype', 'trafficviolationoffensecodes',
       'suspicionoffensecode', 'suspicionsubtype', 'actionstaken',
       'basisforsearch', 'basisforsearchnarrative',
       'basisforpropertyseizure', 'typeofpropertyseized',
       'contrabandorevidence', 'othercontrabanddesc',
       'warningoffensecodes', 'citationoffensecodes', 'resultofstop',
       'infieldciteandreleaseoffensecode', 'custodialarrestoffensecodes',
       'lat', 'v44', 'distancefromcal', 'censustract']]
df['censustract'] = df['censustract'].astype(str)
census_data_multi['censustract'] = census_data_multi['censustract'].astype(str)

df_expanded = pd.merge(df, census_data_multi, on='censustract')
df_expanded.head(2)
df_expanded.shape
df_expanded.columns.unique()
df_expanded.to_csv('C:\\Users\\kharr\\Documents\\GitHub\\Berkeley-PD-ISF-110\\Data\\formatted_allstops(2).csv')

df is (8090, 52) big.


,dateofstop,mm,yyyy,timeofstop,durationofstop,isstopmadeinresponsetocallforser,informationbasedstop,typeofstop,officertypeofassignment,location,...,tract_nonwhitepop,tract_pocpop,tract_aapop,tract_na_aipop,tract_aisianpop,tract_hawaiian,tract_mixed2,tract_totalnumstops,tract_annualstops,tract_distancefromcal
0,2020-10-01,10,2020,11:14,8,False,,Vehicle,"Patrol, traffic enforcement, field operations",2900 block of Shattuck Ave,...,1458.0,1024.0,585.0,11.0,434.0,6.0,422.0,3486,436,1.009877
1,2020-10-02,10,2020,15:21,7,True,,Pedestrian,"Patrol, traffic enforcement, field operations",2900 block of Martin Luther King Jr Way,...,1458.0,1024.0,585.0,11.0,434.0,6.0,422.0,3486,436,1.009877


(8090, 62)

Index(['dateofstop', 'mm', 'yyyy', 'timeofstop', 'durationofstop',
       'isstopmadeinresponsetocallforser', 'informationbasedstop',
       'typeofstop', 'officertypeofassignment', 'location', 'city',
       'islocationak12publicschool', 'ifk12schoolisstopofastudent',
       'schoolname', 'educationcodesection', 'educationcodesubdivision',
       'raceperceivedpriortostop', 'perceivedraceorethnicity', 'gender',
       'perceivedgendernonconforming', 'islgbt', 'perceivedage',
       'personhadlimitedornoenglishfluen', 'perceivedorknowndisability',
       'cityofresidence', 'reasonforstop', 'reasonforstopnarrative',
       'trafficviolationtype', 'trafficviolationoffensecodes',
       'suspicionoffensecode', 'suspicionsubtype', 'actionstaken',
       'basisforsearch', 'basisforsearchnarrative', 'basisforpropertyseizure',
       'typeofpropertyseized', 'contrabandorevidence', 'othercontrabanddesc',
       'warningoffensecodes', 'citationoffensecodes', 'resultofstop',
       'infieldcitea